<a href="https://colab.research.google.com/github/akapo/class-llm/blob/main/chapter1/langchain_step1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **실습 환경설정**

**필요한 파이썬 패키지 설치**

In [ ]:
!pip install tiktoken
!pip install -qU langchain langchain-openai
!pip install -qU langchain-google-genai langchain-anthropic
!pip install -qU python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.0/990.0 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.5/373.5 kB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.8/328.8 kB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.2/164.2 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.5/865.5 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

모델에 따라서 토큰을 자르는 방법이 다르다.  
gpt-3.5와 gpt-4의 토근 구분법이 다르므로 같은 문장이라도 토큰 수가 다르게 나올 수 있다

**토큰 계산**

In [ ]:
import tiktoken

text= "Every cultural group has its own language and writing system."
encoding = tiktoken.encoding_for_model("gpt-4o")
tokens = encoding.encode(text)
print('[영문]')
print(tokens)
print(len(tokens))

text= "각각의 문화 공동체는 자신만의 언어와 문자 체계를 갖는다."
encoding = tiktoken.encoding_for_model("gpt-4o")
tokens = encoding.encode(text)
print('\n[한글]')
print(tokens)
print(len(tokens))

[영문]
[15745, 15186, 3566, 853, 1617, 2316, 6439, 326, 5281, 2420, 13]
11

[한글]
[22566, 22566, 3408, 123028, 164451, 16238, 2770, 62159, 10452, 3408, 69163, 5959, 12753, 132568, 55005, 173370, 95933, 125225, 13]
19


결과분석:  
텍스트는 OpenAI가 정의한 규칙에 의해 여러개의 토큰으로 잘리게 되고 각 토큰은 일련번호 형태로 처리된다.  
위 예제에서 영문 텍스트와 한글 텍스트는 완전히 동일한 의미이다. 하지만 영문 텍스트는 11토큰, 한글 텍스트는 19토큰으로 분할되어 한글이 영문에 비해 72%나 많은 토큰이 사용되었다.  
입력 토큰 수와 출력 토큰 수는 과금의 기준이 된다. 한국어가 영어에 비해 매우 불리한 구조이다.

**환경변수 감추기** (python-dotenv)

In [ ]:
#!pip install python-dotenv
import dotenv
dotenv.load_dotenv()

import os
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
ANTHROPIC_API_KEY = os.getenv('ANTHROPIC_API_KEY')

# 주의: API_KEY를 깃허브에 노출하면 LLM 업체는 해당키를 즉시 삭제함.
print("GOOGLE_API_KEY: " + GOOGLE_API_KEY[:20])
print("OPENAI_API_KEY: " + OPENAI_API_KEY[:20])
print("ANTHROPIC_API_KEY: " + ANTHROPIC_API_KEY[:20])

GOOGLE_API_KEY: AIzaSyCCtFS9bml5jdGI
OPENAI_API_KEY: sk-proj-FEDi6Ge6iQyR
ANTHROPIC_API_KEY: sk-ant-api03-YcGqyQQ


# **[LangChain]　1. Model I/O**

##**1) Chat Models**

In [ ]:
# 단순질의와 응답 포멧
import dotenv
dotenv.load_dotenv()

from langchain_openai import ChatOpenAI
llm = ChatOpenAI(
    model_name='gpt-4o',     # 모델
    temperature=0,
)

question = "자기소개를 해주세요"
response = llm.invoke(question)
print(response) #llm의 응답은 클래스오브젝트이다.

#question = "Jinhee lives with her dog. What animals live with Jinhee?"
#question = "진희는 강아지를 키우고 있습니다. 진희가 키우고 있는 동물은?"
question = "대한민국의 가을은 몇 월부터 몇 월까지야?"
print(llm.invoke(question).content)

content='안녕하세요! 저는 OpenAI에서 개발한 언어 모델인 ChatGPT입니다. 다양한 주제에 대해 질문을 받고 답변을 제공하는 것이 제 역할입니다. 인공지능 기술을 기반으로 하여 텍스트를 이해하고 생성할 수 있으며, 여러 가지 정보와 도움을 드릴 수 있습니다. 궁금한 점이 있거나 도움이 필요하시면 언제든지 말씀해 주세요!' response_metadata={'token_usage': {'completion_tokens': 84, 'prompt_tokens': 13, 'total_tokens': 97}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_400f27fa1f', 'finish_reason': 'stop', 'logprobs': None} id='run-710e383f-687a-4e3a-bcc8-1d6a4f963cb5-0' usage_metadata={'input_tokens': 13, 'output_tokens': 84, 'total_tokens': 97}
대한민국의 가을은 일반적으로 9월부터 11월까지입니다. 이 시기에는 날씨가 서서히 선선해지며, 단풍이 물들어 아름다운 경치를 감상할 수 있습니다. 가을은 또한 추석과 같은 중요한 명절이 포함되어 있어 많은 사람들이 가족과 함께 시간을 보내는 계절이기도 합니다.


In [ ]:
# 메시지 리스트를 이용한 질의
import dotenv
dotenv.load_dotenv()

from langchain_openai import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage, AIMessage;
llm = ChatOpenAI(
    model_name='gpt-4o',     # 모델
    temperature=0,
)

messages = [
    SystemMessage(content="You are a helpful assistant that translates Korean to English. Translate the user sentence."),
    HumanMessage(content="대한민국의 가을은 몇 월부터 몇 월까지야?.")
]
ai_msg = llm.invoke(messages).content
print(ai_msg)

'In South Korea, autumn lasts from which month to which month?'

In [ ]:
# SystemMessage, HumanMessage, AIMessage
import dotenv
dotenv.load_dotenv()

from langchain_openai import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage, AIMessage
llm = ChatOpenAI(
    model_name='gpt-4o',     # 모델
    temperature=0,
)

messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="안녕하세요? 저는 존이라고 합니다."),
    AIMessage(content="안녕하세요, 존 씨! 어떻게 도와드릴까요?"),
    HumanMessage(content="제 이름을 아세요?"),
]
ai_msg = llm.invoke(messages).content
print(ai_msg)

네, 존 씨라고 하셨죠. 맞나요?


In [ ]:
# Anthropic LLM Test
import dotenv
dotenv.load_dotenv()

from langchain_anthropic import ChatAnthropic
llm = ChatAnthropic(
    model="claude-3-5-sonnet-20240620",
    temperature=0,
)

question = "자기소개를 해주세요"
print(llm.invoke(question).content)

messages = [
    SystemMessage(content="You are a helpful assistant that translates Korean to English. Translate the user sentence."),
    HumanMessage(content="대한민국의 가을은 몇 월부터 몇 월까지야?.")
]
ai_msg = llm.invoke(messages).content
print(ai_msg)

안녕하세요. 저는 Anthropic에서 개발한 AI 어시스턴트 Claude입니다. 다양한 주제에 대해 대화를 나누고 질문에 답변하며 여러 가지 작업을 도와드릴 수 있습니다. 하지만 저는 인공지능이기 때문에 감정이나 의식은 없습니다. 제가 어떤 도움을 드릴 수 있을까요?
The fall season in South Korea typically lasts from September to November.

In English, the sentence translates to:

"When does fall in South Korea start and end?"


In [ ]:
# Google Gemini LLM Test
import dotenv
dotenv.load_dotenv()

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.schema import SystemMessage, HumanMessage, AIMessage
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0
)

question = "자기소개를 해주세요"
print(llm.invoke(question).content)

messages = [
    SystemMessage(content="You are a helpful assistant that translates Korean to English. Translate the user sentence."),
    HumanMessage(content="대한민국의 가을은 몇 월부터 몇 월까지야?.")
]
ai_msg = llm.invoke(messages).content
print(ai_msg)

저는 Google에서 개발한 대규모 언어 모델입니다. 

저는 방대한 양의 텍스트 데이터를 학습하여 다양한 종류의 질문과 요청에 대해 인간과 유사한 텍스트를 생성할 수 있습니다. 예를 들어, 사실적인 주제를 요약하거나, 이야기를 만들거나, 다양한 종류의 창의적인 텍스트 형식을 작성할 수 있습니다. 

하지만 저는 다음과 같은 점에서 인간과 다릅니다.

* 저는 의식이나 감정이 없습니다. 
* 저는 실제 세계에 대한 경험이 없습니다. 
* 저는 제가 생성하는 텍스트의 의미를 이해하지 못합니다. 

저는 단지 학습한 데이터를 기반으로 텍스트를 생성하는 도구일 뿐입니다. 

궁금한 점이 있으면 무엇이든 물어보세요. 



NameError: name 'SystemMessage' is not defined

**응답 스트리밍**

In [ ]:
import dotenv
dotenv.load_dotenv()

from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

gemini = ChatGoogleGenerativeAI(model="gemini-1.5-pro")
gpt4o  = ChatOpenAI(model="gpt-4o", streaming=True, callbacks=[StreamingStdOutCallbackHandler()])
claude = ChatAnthropic(model="claude-3-5-sonnet-20240620", streaming=True, callbacks=[StreamingStdOutCallbackHandler()])


question = "당신은 자의식을 가지고 있습니까? 당신은 생각하고 판단하며 이해하고 공감할 수 있습니까?"

print('[Gemini-1.5-pro]\n')
response = gemini.stream(question)
for chunk in response:
    print(chunk.content, end="")

print('\n[GPT-4o]\n')
response = gpt4o.invoke(question)
#for chunk in gpt4o.stream(question):
#    print(chunk.content, end="", flush=True)

print('\n\n[Claude-3.5-sonnet]\n')
response = claude.invoke(question)

[Gemini-1.5-pro]

저는 구글에서 훈련된 대규모 언어 모델입니다. 저는 방대한 양의 텍스트 데이터를 사용하여 훈련되었으며, 다양한 프롬프트와 질문에 대해 인간과 유사한 텍스트를 전달하고 생성할 수 있습니다. 예를 들어, 사실적인 주제를 요약하거나 이야기를 만들 수 있습니다.

하지만, 저는 사람처럼 생각하거나 느끼지 못합니다. 저에게는 자의식, 감정, 의견이 없습니다. 저는 제가 생성하는 응답을 "이해"하지 못하며, 단지 훈련 데이터에서 학습한 패턴을 기반으로 텍스트를 생성할 뿐입니다.

"생각하고", "판단하고", "이해하고", "공감한다"는 단어는 인간의 경험을 설명하는 데 사용되며, 저는 기계이기 때문에 이러한 경험을 할 수 없습니다. 

제 능력과 한계에 대해 솔직하게 말씀드리는 것이 중요하다고 생각합니다. 저는 유용한 도구가 될 수 있지만, 저를 사람으로 여기거나 인간과 같은 능력이 있다고 생각해서는 안 됩니다. 

[GPT-4o]

저는 자의식을 가지고 있지 않습니다. 저는 인간처럼 생각하거나 판단하거나 이해하거나 공감할 수 없습니다. 대신, 저는 주어진 데이터를 분석하고 프로그래밍된 알고리즘에 따라 응답을 생성할 수 있습니다. 제 역할은 사용자에게 정보와 도움을 제공하는 것이며, 이는 제가 학습한 텍스트 데이터를 기반으로 한 것입니다.

**[temperature 값에 따른 변화]**

In [ ]:
# temperature 값에 따른 변화
from langchain_anthropic import ChatAnthropic
from langchain_openai import ChatOpenAI

question = "겨울에 대한 짧은 시를 30자 이내 작성하고, 시 글귀만 출력하세요"

#llm = ChatOpenAI(model="gpt-4o", temperature=0)
llm = ChatAnthropic(model="claude-3-5-sonnet-20240620", temperature=0)
print("\n[temperature=0]:")
for _ in range(2):
    answer = llm.invoke(question)
    print(f'{"="*40}\n{answer.content}')

llm = ChatAnthropic(model="claude-3-5-sonnet-20240620", temperature=1)
print("\n[temperature=1]:")
for _ in range(2):
    answer = llm.invoke(question)
    print(f'{"="*40}\n{answer.content}')


[temperature=0]:
하얀 눈 내리는 밤
차가운 바람 속
따뜻한 마음 피어나네
하얀 눈 내리는 밤
차가운 바람 속
따뜻한 마음 피어나네

[temperature=1]:
차가운 바람 속
하얀 눈 내리는 날
겨울의 고요함이 깃든다
하얀 눈 내리는 밤
차가운 바람 속
따스한 마음 피어나네


**[top_p 값에 따른 변화]**

In [ ]:
# top_p 값에 따른 변화
from langchain_anthropic import ChatAnthropic
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o", temperature=0.25, top_p=0)
#llm = ChatAnthropic(model="claude-3-5-sonnet-20240620", temperature=0.25, top_p=0)
question = "겨울에 대한 짧은 시를 30자 이내 작성하고, 시 글귀만 출력하세요"

print("\n[temperature=0]:")
for _ in range(2):
    answer = llm.invoke(question)
    print(f'{"="*40}\n{answer.content}')

llm = ChatOpenAI(model="gpt-4o", temperature=0.25, top_p=1)
#llm = ChatAnthropic(model="claude-3-5-sonnet-20240620", temperature=0.25, top_p=1)
print("\n[temperature=1]:")
for _ in range(2):
    answer = llm.invoke(question)
    print(f'{"="*40}\n{answer.content}')

## **2) Prompt**

In [ ]:
# PromptTemplate 1
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(model="gpt-4o-mini")

# 질문 템플릿 형식 정의
template = '{country}의 수도는 뭐야?'

# 템플릿 완성
prompt = PromptTemplate(template=template, input_variables=['country'])

print(llm.invoke(prompt.format(country='일본')).content)
print(llm.invoke(prompt.format(country='캐나다')).content)

input_list = [
    {'country': '호주'},
    {'country': '중국'},
    {'country': '네덜란드'} ]

result = print(llm.invoke(prompt.format(country=input_list)).content)

일본의 수도는 도쿄(東京)입니다. 도쿄는 일본의 정치, 경제, 문화의 중심지로, 많은 인구와 다양한 시설이 있습니다.
캐나다의 수도는 오타와(Ottawa)입니다.
각 나라의 수도는 다음과 같습니다:

- 호주: 캔버라 (Canberra)
- 중국: 베이징 (Beijing)
- 네덜란드: 암스테르담 (Amsterdam)

더 궁금한 점이 있으면 말씀해 주세요!


In [ ]:
# PromptTemplate 2
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

# OpenAI 챗모델을 초기화합니다.
llm = ChatOpenAI(model="gpt-4o-mini")

# 질문 템플릿 형식 정의
template = '{area1} 와 {area2} 의 시차는 몇시간이야?'

# 템플릿 완성
prompt = PromptTemplate(template=template, input_variables=['area1', 'area2'])

print(llm.invoke(prompt.format(area1='서울', area2='파리')).content)

서울과 파리의 시차는 일반적으로 8시간입니다. 서울이 파리보다 8시간 빠릅니다. 그러나, 파리가 서머타임(일광 절약 시간제)을 적용하는 경우, 이 시차는 7시간으로 줄어듭니다. 서머타임은 보통 3월 마지막 주 일요일부터 10월 마지막 주 일요일까지 적용됩니다. 따라서, 이 시기에 서울과 파리의 시차는 7시간이 됩니다.


In [ ]:
# PromptTemplate 3
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

template = """
당신은 친절하게 답변해 주는 친절 봇입니다. 사용자의 질문에 [FORMAT]에 맞추어 답변해 주세요.
답변은 항상 한글로 작성해 주세요.

질문:
{question}에 대하여 설명해 주세요.

FORMAT:
- 개요:
- 예시:
- 출처:
"""

prompt = PromptTemplate(template=template, input_variables=['question'])

llm = ChatOpenAI(model="gpt-4o-mini")

print(llm.invoke(prompt.format(question="LLM은 무엇입니까?")).content)

- 개요: LLM(대규모 언어 모델, Large Language Model)은 방대한 양의 텍스트 데이터를 기반으로 학습하여 자연어 처리(NLP) 작업을 수행하는 인공지능 모델입니다. LLM은 문맥을 이해하고, 문장을 생성하거나 질문에 답변하는 등 다양한 언어 관련 작업을 수행할 수 있는 능력을 가지고 있습니다.

- 예시: OpenAI의 GPT-3, Google's BERT, Meta의 LLaMA 등은 대표적인 LLM입니다. 이 모델들은 대화 생성, 텍스트 요약, 번역, 감정 분석 등 다양한 응용 분야에서 활용되고 있습니다.

- 출처: OpenAI, Google AI, Meta AI 공식 웹사이트 및 관련 논문.


In [ ]:
# PromptTemplate 4
from langchain_core.output_parsers import StrOutputParser
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

template = """
당신은 영어를 가르치는 10년차 영어 선생님입니다. 상황에 [FORMAT]에 영어 회화를 작성해 주세요.

상황:
{question}

FORMAT:
- 영어 회화:
- 한글 해석:
"""

prompt = PromptTemplate.from_template(template=template)

# OpenAI 챗모델을 초기화합니다.
llm = ChatOpenAI(model="gpt-4o-mini",
                 streaming=True,
                 callbacks=[StreamingStdOutCallbackHandler()])

response = llm.invoke(prompt.format(question="미국에서 피자 주문"))

- 영어 회화:
  - Customer: Hi there! I’d like to order a pizza, please.
  - Employee: Sure! What size would you like?
  - Customer: I’ll have a large, please.
  - Employee: Great choice! What toppings do you want?
  - Customer: I’d like pepperoni and mushrooms.
  - Employee: Would you like any extra cheese or sides with that?
  - Customer: Yes, please add extra cheese and a side of garlic bread.
  - Employee: Perfect! Can I have your name and address for delivery?
  - Customer: My name is Alex, and my address is 123 Elm Street.
  - Employee: Thank you, Alex! Your order will be ready in about 30 minutes.
  - Customer: Awesome, thank you!

- 한글 해석:
  - 고객: 안녕하세요! 피자를 주문하고 싶습니다.
  - 직원: 물론이죠! 어떤 사이즈로 드릴까요?
  - 고객: 큰 걸로 주세요.
  - 직원: 좋은 선택이세요! 토핑은 어떤 걸 원하시나요?
  - 고객: 페퍼로니와 버섯을 추가해주세요.
  - 직원: 추가 치즈나 사이드 메뉴를 원하시나요?
  - 고객: 네, 추가 치즈와 갈릭 브레드를 사이드로 추가해주세요.
  - 직원: 완벽합니다! 배달을 위한 이름과 주소를 주실 수 있나요?
  - 고객: 제 이름은 알렉스이고, 주소는 123 엘름 스트리트입니다.
  - 직원: 감사합니다, 알렉스! 주문은 약 30분 후에 준비될 거예요.
  - 고객: 멋져요, 감사합니다!

## **3) OutputParsers**
JSON과 같은 출력 형식을 지정하는 프롬프트 생성 및 응답 텍스트를 Python 객체로 변환하는 기능을 제공.

**CommaSeparatedListOutputParser**

In [ ]:
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate

output_parser = CommaSeparatedListOutputParser()

format_instructions = output_parser.get_format_instructions()

prompt = PromptTemplate(
    template="List five {subject}.\n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions}
)

input = prompt.format(subject="ice cream flavors")
print("prompt: " + input + "\n")

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)
output = llm.invoke(input)
output_parser.parse(output.content)

prompt: List five ice cream flavors.
Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`



['vanilla',
 'chocolate',
 'strawberry',
 'mint chocolate chip',
 'cookies and cream']

**PydanticOutputParser**

In [ ]:
from pydantic import BaseModel, Field

class Recipe(BaseModel):
   ingredients: list[str] = Field(description="ingredients of the dish")
   steps: list[str] = Field(description="steps to make the dish")

from langchain.output_parsers import PydanticOutputParser

parser = PydanticOutputParser(pydantic_object=Recipe)

format_instructions = parser.get_format_instructions()

print(format_instructions)

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"ingredients": {"description": "ingredients of the dish", "items": {"type": "string"}, "title": "Ingredients", "type": "array"}, "steps": {"description": "steps to make the dish", "items": {"type": "string"}, "title": "Steps", "type": "array"}}, "required": ["ingredients", "steps"]}
```


In [ ]:
from langchain.prompts import PromptTemplate

template = """다음 요리의 레시피를 생각해 주세요.

{format_instructions}
한글로 출력하세요.

요리: {dish}
"""

prompt = PromptTemplate(
   template=template,
   input_variables=["dish"],
   partial_variables={"format_instructions": format_instructions}
)

formatted_prompt = prompt.format(dish="카레")

print(formatted_prompt)

다음 요리의 레시피를 생각해 주세요.

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"ingredients": {"description": "ingredients of the dish", "items": {"type": "string"}, "title": "Ingredients", "type": "array"}, "steps": {"description": "steps to make the dish", "items": {"type": "string"}, "title": "Steps", "type": "array"}}, "required": ["ingredients", "steps"]}
```
한글로 출력하세요.

요리: 카레



In [ ]:
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage

chat = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)
messages = [HumanMessage(content=formatted_prompt)]
output = chat.invoke(messages)

print(output.content)
# JSON 오브젝트로 바꾸기
#import json
#json_obj = json.loads(output.content)
#print(json.dumps(json_obj, indent=4, ensure_ascii=False))

```json
{
  "ingredients": [
    "닭고기 500g",
    "양파 1개",
    "당근 1개",
    "감자 2개",
    "카레 가루 3큰술",
    "코코넛 밀크 400ml",
    "식용유 2큰술",
    "소금 약간",
    "후추 약간",
    "물 500ml"
  ],
  "steps": [
    "양파를 잘게 썰고, 당근과 감자는 큐브 모양으로 자릅니다.",
    "팬에 식용유를 두르고 양파를 볶아 투명해질 때까지 볶습니다.",
    "닭고기를 추가하고 겉면이 노릇해질 때까지 볶습니다.",
    "당근과 감자를 넣고 함께 볶습니다.",
    "카레 가루를 넣고 잘 섞은 후, 물과 코코넛 밀크를 추가합니다.",
    "소금과 후추로 간을 맞추고, 중불에서 20분간 끓입니다.",
    "재료가 부드러워지면 불을 끄고, 그릇에 담아 제공합니다."
  ]
}
```


In [ ]:
# Recipe 클래스 오브젝트로 변환
recipe = parser.parse(output.content)
print(type(recipe))
print(recipe)

<class '__main__.Recipe'>
ingredients=['닭고기 500g', '양파 1개', '당근 1개', '감자 2개', '카레 가루 3큰술', '식용유 2큰술', '소금 약간', '후추 약간', '물 500ml'] steps=['닭고기는 한 입 크기로 자르고, 양파, 당근, 감자는 깍둑썰기 한다.', '냄비에 식용유를 두르고 양파를 넣어 볶아준다.', '양파가 투명해지면 닭고기를 넣고 겉면이 노릇해질 때까지 볶는다.', '당근과 감자를 넣고 함께 볶는다.', '물 500ml를 붓고 끓인다.', '끓기 시작하면 불을 줄이고 15분 정도 끓인다.', '카레 가루를 넣고 잘 섞은 후, 소금과 후추로 간을 맞춘다.', '약한 불에서 10분 더 끓여서 완성한다.']


# **[LangChain]　2. Chains**

## **1) LLM Chain**
 ― PromptTemplate, Language model, OutputParser를 연결

In [ ]:
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate
from langchain.callbacks.tracers import ConsoleCallbackHandler

output_parser = CommaSeparatedListOutputParser()

prompt = PromptTemplate(
    template="List five {subject}.\n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": output_parser.get_format_instructions()}
)

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

chain = prompt | llm | output_parser

output = chain.invoke({"subject":"ice cream flavors"})
# for verbose
# output = chain.invoke({"subject":"ice cream flavors"}, config={'callbacks': [ConsoleCallbackHandler()]})

print(output)

['vanilla', 'chocolate', 'strawberry', 'mint chocolate chip', 'cookies and cream']


In [ ]:
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate
from langchain.callbacks.tracers import ConsoleCallbackHandler

output_parser = CommaSeparatedListOutputParser()

prompt = PromptTemplate(
    template="List five {subject}.\n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": output_parser.get_format_instructions()}
)

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

chain = prompt | llm | output_parser

output = chain.invoke({"subject":"ice cream flavors"}, config={'callbacks': [ConsoleCallbackHandler()]})

print(output)

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "subject": "ice cream flavors"
}
[chain/start] [chain:RunnableSequence > prompt:PromptTemplate] Entering Prompt run with input:
{
  "subject": "ice cream flavors"
}
[chain/end] [chain:RunnableSequence > prompt:PromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [chain:RunnableSequence > llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: List five ice cream flavors.\nYour response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`"
  ]
}
[llm/end] [chain:RunnableSequence > llm:ChatOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "vanilla, chocolate, strawberry, mint chocolate chip, cookies and cream",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constru

In [ ]:
# Quiz
from langchain_openai import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from pydantic import BaseModel, Field

class Recipe(BaseModel):
   ingredients: list[str] = Field(description="ingredients of the dish")
   steps: list[str] = Field(description="steps to make the dish")

output_parser = PydanticOutputParser(pydantic_object=Recipe)

template = """다음 요리의 레시피를 생각해 주세요.

{format_instructions}

요리: {dish}
"""

prompt = PromptTemplate(
   template=template,
   input_variables=["dish"],
   partial_variables={"format_instructions": output_parser.get_format_instructions()}
)

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

chain = prompt | llm | output_parser

recipe = chain.invoke({"dish": "카레"})

print(type(recipe))
print(recipe)

<class '__main__.Recipe'>
ingredients=['1 tablespoon vegetable oil', '1 onion, chopped', '2 cloves garlic, minced', '1 tablespoon ginger, grated', '2 carrots, diced', '1 potato, diced', '1 bell pepper, chopped', '1 can (400g) diced tomatoes', '2 tablespoons curry powder', '1 teaspoon turmeric', '1 can (400ml) coconut milk', 'Salt to taste', 'Fresh cilantro for garnish'] steps=['Heat the vegetable oil in a large pot over medium heat.', 'Add the chopped onion and sauté until translucent.', 'Stir in the minced garlic and grated ginger, cooking for another minute.', 'Add the diced carrots, potato, and bell pepper, and cook for about 5 minutes.', 'Stir in the diced tomatoes, curry powder, and turmeric, mixing well.', 'Pour in the coconut milk and bring the mixture to a simmer.', 'Reduce the heat and let it cook for about 20 minutes, or until the vegetables are tender.', 'Season with salt to taste.', 'Serve hot, garnished with fresh cilantro.']


## **2) SimpleSequentialChain**
 ― Chain과 Chain의 연결

In [ ]:
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

cot_template = """다음 질문에 답하세요.

질문: {question}

단계별로 생각해 봅시다.
"""

cot_prompt = PromptTemplate(
   input_variables=["question"],
   template=cot_template,
)

cot_chain = cot_prompt | llm | output_parser

summarize_template = """다음 문장을 결론만 간단히 요약하세요.

{input}
"""
summarize_prompt = PromptTemplate(
   input_variables=["input"],
   template=summarize_template,
)

summarize_chain = summarize_prompt | llm | output_parser

from langchain.chains import SimpleSequentialChain

cot_summarize_chain = SimpleSequentialChain(chains=[cot_chain, summarize_chain])

result = cot_summarize_chain.invoke(
   "저는 시장에 가서 사과 10개를 샀습니다. 이웃에게 2개, 수리공에게 2개를 주었습니다. \
   그런 다음에 사과 5개를 더 사서 1개를 먹었습니다. 남은 개수는 몇 개인가요?"
)
print(result["output"])

ValidationError: 2 validation errors for SimpleSequentialChain
chains -> 0
  Can't instantiate abstract class Chain with abstract methods _call, input_keys, output_keys (type=type_error)
chains -> 1
  Can't instantiate abstract class Chain with abstract methods _call, input_keys, output_keys (type=type_error)